In [1]:
import os
import pandas as pd
import xlrd
from pathlib import Path
import openpyxl
from openpyxl import load_workbook
import numpy as np
import re
import ast
from xls2xlsx import XLS2XLSX
from csv import writer

In [2]:
#Function to extract 1045 sheet name from xlsx file only

def find1045_xlsx(filepath):
    #read file
    xlsx_file = Path(filepath)
    wb_obj = openpyxl.load_workbook(xlsx_file)

    #get sheet names
    sheetnames=wb_obj.sheetnames

    #Create a keywords list to identify ABS 1045 forms
    keywords= ["common", "procedures", "1045", "form"]

    #Search for the sheet by keyword
    for name in sheetnames:
        wholename=name.lower()
        wordsInName=wholename.split()
        match =  any(item in wordsInName for item in keywords)
        if match==True:
            sheetname=(wb_obj[name])
            return(sheetname)
            break


In [3]:
#Function to extract all dataframes from a single xlsx file
def getdata_xlsx(file):
    dflist=[]
    coordlist=[]
    sheet=find1045_xlsx(file)
    alphabet=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
    found=False
    keywords1 = ["evaluation", "management", "services"]
    keywords2 = ["laboratory", "pathology", "services"]
    keywords3= ["radiology", "services"]
    keywords4 = ["medicine", "services"]
    keywords5 = ["surgery", "services"]
    keywords6 = ["other", "common", "outpatient", "procedures"]
    keywords7 = ["count", "of", "reported", "procedures",]
    keywordslists=[keywords1,keywords2,keywords3,keywords4,keywords5,keywords6,keywords7]
    for wordlist in keywordslists:
        for row in sheet.iter_rows():
                for cell in row:
                    heading = str(cell.value).lower()
                    wordsInHeading=heading.split()
                    match =  all(item in wordsInHeading for item in wordlist)
                    if match==True:
                        coord1=cell.coordinate
                        coordlist.append(coord1)
                        break
    for coordinate in coordlist[0:-1]:
        splitcoord=re.split('(\d+)', coordinate)
        nextbreak=coordlist[int(coordlist.index(coordinate)+1)]
        splitbreak=re.split('(\d+)', nextbreak)
        lastcell=f"{int(splitbreak[1])-1}"
        coord2=f"C{lastcell}"

        cellgroup=[coordinate,coord2]
        df=pd.DataFrame(sheet[f"{cellgroup[0]}:{cellgroup[1]}"])



        dflist.append(df)

    return(dflist)

  
    

In [4]:
#Function to convert cell object values to plaintext values
def convertvalue(x):
    return x.value

In [5]:
####THIS ONE
#GET THE ENTIRE MASTERLIST FROM ONE FILE, AS A FUNCTION
def getMaster(somefile):
        sheet=find1045_xlsx(somefile)
        masterlist=[]
        rawlist=getdata_xlsx(somefile)
        for i in range(len(rawlist)):
                df=rawlist[i].applymap(convertvalue)
                raw_name=sheet["A1"].value
                hos_name=raw_name.replace('Hospital Name: ', '')
                df["Hospital"]=hos_name
                df.columns = df.iloc[0]
                df["Service Type"]=df.iloc[0,0]
                df.rename(columns={ df.columns[0]: "Service" }, inplace = True)
                df = df.iloc[1: , :]
                        #add the hospital name as a variable
                        
                masterlist.append(df)
                return(masterlist)

In [6]:
#Get all the data, put it in a dataframe list, then convert all dataframes from a single sheet into plaintext, returning the final list
def get1045dataframes(xlsx_file):
    masterlist=[]
    masterdf=pd.DataFrame()    
    rawlist=getdata_xlsx(xlsx_file)
    #Get the column names from the dataframe
    for i in range(len(masterlist)):
        if len(df[i])>0:
            masterlist[i].columns = df.iloc[0]
            masterlist[i]["Service Type"]=df.iloc[0,0]
            masterlist[i].rename(columns={ df.columns[0]: "Service" }, inplace = True)
            masterlist[i]=masterlist[i].drop(df.index[0], inplace=True)
            raw_name=sheet["A1"].value
            hos_name=raw_name.replace('Hospital Name: ', '')
            masterlist[i]["Hospital"]=hos_name
            df=rawlist[i].applymap(convertvalue)       
            masterlist.append(df)
            masterdf.append(masterlist)

    return(masterdf)

In [7]:
#Get the column names from the dataframe
for df in masterlist:
    if len(df)>0:
        df.columns = df.iloc[0]
        df["Service Type"]=df.iloc[0,0]
        df.rename(columns={ df.columns[0]: "Service" }, inplace = True)
        df=df.drop(df.index[0], inplace=True)




NameError: name 'masterlist' is not defined

In [24]:
#The big final block of code that will probably crash your computer
ultradf=pd.DataFrame()
#get the names of all files from each directory, then test if each file has a 1045 in it.
dirlist = list(os. listdir())
for i in range(len(dirlist)):
    os.chdir(dirlist[i])
    for root, dirs, files in os.walk("."):
        for filename in files:
            # try:
            sheet=find1045_xlsx(filename)
            masterdf=get1045dataframes(sheet)
            ultradf.append(masterdf)
            print(dirlist[i])
    #         except:
    #             print(f"{filename} is the wrong data type: skipping")
    # os.chdir(chargemaster)


TypeError: expected str, bytes or os.PathLike object, not Worksheet

In [8]:
#Function to convert all xls files to xlsx, removing the original file
def convert(filename):
    p.save_book_as(file_name=f"{filename}",
                dest_file_name=f"{filename[:-4]}.xlsx" )

In [9]:
#function to rstrip better
def cleanse_func(v):
     v = v.replace("&nbsp;", "")
     v = v.rstrip().strip()
     return v


In [10]:
#A function to convert all files in subfolders starting from the absolute filepath of the year's chargemaster folder

def convertfrom(directory):
    #Use absolute path to move to the directory containing all hospital folders
    os.chdir(directory)
    #save current working directory to a variable
    cwd=os.getcwd()
    #Convert all files to xlsx format
    dirlist = list(os. listdir())
    for i in range(len(dirlist)):
        os.chdir(dirlist[i])
        for root, dirs, files in os.walk("."):
            for filename in files:
                if filename[-3:]=="xls":
                    try:
                        convert(filename)
                        os.remove(filename)
                    except:
                        print(filename)
                else:
                    continue
        os.chdir("..")
    os.chdir(cwd)

In [21]:
convertfrom("/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster")

106364188_CDM_2020.xls
106364188_CDM_2020-Cloud.xls
106332172_CDM_2020.xls
106332172_CDM_2020-Cloud.xls
106201281_PCT_CHG_2020.xls
